# Quantitative Momentus Strategy

Momentum investing means a system of buying stocks that gave highest returns over some period of time.

Ths, we will create a python script to provide for an investing strategy selecting 50 stocks with the highest price momentum changes. later, we will provide equal weight recomended portfolio trade consisting of these 50 highest momentum stocks.

In [23]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
#from scipy.stats import percentileofscore as score
from scipy import stats #- stats.percentileofscorecentileofscore

## Importing the list of stocks

The constituents of S&P index keeps on changing and thus needs to be updated overtime. We have used a list provided in the reference.

In [2]:
stocks = pd.read_csv('./data/sp_500_stocks.csv')

# Acquiring an API Token

IEX Cloud API is the data provider that we will be using in thsi script. We need to import IEX Cloud API token to be used in this script. 

We will be using a free, sand box API token for this course and thus the data generated using this token will not match with the actual data. For getting actual data, we need to go for a paid plan. 

In [3]:
from data.secrets import IEX_CLOUD_API_TOKEN

In quantitative momentum strategy, initially we will be using 1 year price return to finalise the portfolio. 

Later we will be using following price changes to finalise the portfolio:
- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns

In [4]:
base_url = "https://sandbox.iexapis.com/stable"
symbol = "AAPL"
api_url = f'{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

In [5]:
data = requests.get(api_url).json()
#print(type(request))
#print(request.status_code)
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ')S (QGKTAOLNSETEDAAAGLNC/MS  BLRE',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'affiocli',
 'close': None,
 'closeTime': None,
 'closeSource': 'aifclifo',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 124.92,
 'latestSource': 'Close',
 'latestTime': 'December 4, 2020',
 'latestUpdate': 1654794999189,
 'latestVolume': None,
 'iexRealtimePrice': 126.93,
 'iexRealtimeSize': 104,
 'iexLastUpdated': 1643581640625,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 127.09,
 'previousVolume': 81142023,
 'change': -0.7,
 'changePercent': -0.00562,
 'volume': None,
 'iexMarketPercent': 0.0016646975301556188,
 'iexVolume': 1410,
 'a

In [6]:
base_url = "https://sandbox.iexapis.com/stable"
symbol = "AAPL"
api_url_stats = f'{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

In [7]:
data_stats = requests.get(api_url_stats).json()
data_stats

{'companyName': 'Apple Inc',
 'marketcap': 2100202479495,
 'week52high': 136.3,
 'week52low': 58.47,
 'week52change': 0.8339938634162407,
 'sharesOutstanding': 17071666948,
 'float': 0,
 'avg10Volume': 111137318,
 'avg30Volume': 102820607,
 'day200MovingAvg': 108.26,
 'day50MovingAvg': 121.69,
 'employees': 0,
 'ttmEPS': 3.3,
 'ttmDividendRate': 0.8339483372777458,
 'dividendYield': 0.00674588014700318,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-26',
 'nextEarningsDate': '0',
 'peRatio': 37.71936513210303,
 'beta': 1.1949410438961157,
 'maxChangePercent': 46.765887935307745,
 'year5ChangePercent': 3.4795471574375973,
 'year2ChangePercent': 1.9044579680324025,
 'year1ChangePercent': 0.8303528213795425,
 'ytdChangePercent': 0.6971762370247483,
 'month6ChangePercent': 0.494517066666445,
 'month3ChangePercent': 0.012967206967060416,
 'month1ChangePercent': 0.030277167064103213,
 'day30ChangePercent': 0.03097044868398766,
 'day5ChangePercent': 0.04979640951181011}

In [8]:
print(data["latestPrice"])
print(data_stats["year1ChangePercent"])

124.92
0.8303528213795425


### Here, we will be using One-year Price return to get the portfolio.

In [9]:
my_columns = ['Tickers', 'Stock Price', 'One-Year Price Return', 'Number of Stocks to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Tickers,Stock Price,One-Year Price Return,Number of Stocks to Buy


In [10]:
final_dataframe = final_dataframe.append(
    pd.Series([
        symbol,
        data["latestPrice"],
        data_stats["year1ChangePercent"],
        'N/A'
    ],
        index = my_columns),
    ignore_index = True)
    
final_dataframe

,Tickers,Stock Price,One-Year Price Return,Number of Stocks to Buy
0,AAPL,124.92,0.830353,N/A


# Loop through tickers in S&P stock list

Here we will loop through all the tickers in the S&P stock list using a for loop. We can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop.We will pull all the price and market cap data for S&P stocks and save all these data in a panda data frame. Since the call will be serial one and so it will take time to get the result.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol in stocks['Ticker'][:5]:
    api_url_quote = f'{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    api_url_stats = f'{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
    data_quote = requests.get(api_url_quote).json()
    data_stats = requests.get(api_url_stats).json()
    final_dataframe = final_dataframe.append(
                                    pd.Series([
                                        symbol,
                                        data_quote["latestPrice"],
                                        data_stats["year1ChangePercent"],
                                        'N/A'
                                        ],
                                        index = my_columns),
                                    ignore_index = True)
final_dataframe    

,Tickers,Stock Price,One-Year Price Return,Number of Stocks to Buy
0,A,115.22,0.428594,N/A
1,AAL,16.40,-0.426421,N/A
2,AAP,156.83,-0.007530,N/A
3,AAPL,123.98,0.828896,N/A
4,ABBV,107.61,0.312000,N/A


# Using batch API call to improve performance

Making individual HTTP call for individual stoc is very time consuming as executing http request is one of the slowest thing in python. Looping through all the individual 500 stocks is going to be really very slow. Thus, we will be using Batch API calls to improve the response time of the code. 

For this we will divide the calls into chunks with each chunk containing 100 stocks. 

In [12]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

Here we divide all the stocks into chunks where each chunk is a list of 100 stocks.

In [72]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

#print(symbol_groups)
    
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

Now we will make batch api call for each chunk. In this case there will be only 6 call for getting the response for all the stocks. Since rresponse time for batch call is less so it will take less time to get response in batch call.

In [14]:
final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'{base_url}/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        #print(symbol)
        final_dataframe = final_dataframe.append(
                                    pd.Series([
                                        symbol,
                                        data[symbol]['quote']['latestPrice'],
                                        data[symbol]['stats']['year1ChangePercent'],
                                        'N/A'
                                        ],
                                        index = my_columns),
                                    ignore_index = True)
final_dataframe


,Tickers,Stock Price,One-Year Price Return,Number of Stocks to Buy
0,A,119.260,0.424756,N/A
1,AAL,17.000,-0.413956,N/A
2,AAP,153.390,-0.00773319,N/A
3,AAPL,123.120,0.83152,N/A
4,ABBV,110.840,0.309741,N/A
...,...,...,...,...
500,YUM,107.080,0.0949022,N/A
501,ZBH,155.150,0.0117203,N/A
502,ZBRA,395.110,0.488775,N/A
503,ZION,45.187,-0.0814446,N/A


# Removing low momentum stocks from Data frame

We will be following a investment strategy where we will identify the highest 50 momentum stocks in the S&P 500.

Thus, we will remove all the stocks in our DataFrame that are below this momentum threshold. For this, we need to sort the DataFrame by the stocks' one-year price return, and drop all stocks which are outside top 50.

In [15]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Tickers,Stock Price,One-Year Price Return,Number of Stocks to Buy
0,CARR,39.50,2.18144,N/A
1,NVDA,545.85,1.62704,N/A
2,AMD,97.20,1.42506,N/A
3,ALB,140.24,1.21631,N/A
4,LB,38.72,1.19276,N/A
5,FCX,25.42,1.16569,N/A
6,PYPL,222.62,1.12504,N/A
7,QCOM,164.49,0.942653,N/A
8,NOW,539.50,0.938291,N/A
9,FDX,299.43,0.93303,N/A


# Find the number of shares to buy

Now we will calculate the number of share to buy in an quantitative momentum strategy. For this we first need to take input for the portfolio size from the user. 

In [16]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


Now based on the portfolio size, we will be calculating the number of shares that need to purchased in a top 50 quantitative momentum stocks strategy.

In [17]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Stocks to Buy'] = math.floor(position_size/final_dataframe['Stock Price'][i])

final_dataframe

/Users/onkar.mishra/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Tickers,Stock Price,One-Year Price Return,Number of Stocks to Buy
0,CARR,39.50,2.18144,496
1,NVDA,545.85,1.62704,35
2,AMD,97.20,1.42506,201
3,ALB,140.24,1.21631,139
4,LB,38.72,1.19276,506
5,FCX,25.42,1.16569,771
6,PYPL,222.62,1.12504,88
7,QCOM,164.49,0.942653,119
8,NOW,539.50,0.938291,36
9,FDX,299.43,0.93303,65


# Build a Better (and More Realistic) Momentum Strategy¶

In a real world scenario, the momentum stocks do not depend only on 1-year price return. In reality, they depend on returns for different time period i.e shorter and longer periods both. 

Thus, to identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns

Thus, we will create a high quality momentum score based on price returns and percentile score of stocks in each of the above 4 time periods. 

Accordingly, we will create DataFrame based on the above 4 time periods.

In [61]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns= hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'{base_url}/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                        pd.Series([
                                symbol, 
                                data[symbol]['quote']['latestPrice'],
                                'N/A',
                                data[symbol]['stats']['year1ChangePercent'],
                                'N/A',
                                data[symbol]['stats']['month6ChangePercent'],
                                'N/A',
                                data[symbol]['stats']['month3ChangePercent'],
                                'N/A',
                                data[symbol]['stats']['month1ChangePercent'],
                                'N/A',
                                'N/A'                          
                        ],
                        index = hqm_columns),
                    ignore_index = True)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.710,N/A,0.433324,N/A,0.289282,N/A,0.200653,N/A,0.0233394,N/A,N/A
1,AAL,16.742,N/A,-0.406301,N/A,-0.122349,N/A,0.211873,N/A,0.452247,N/A,N/A
2,AAP,151.820,N/A,-0.00761468,N/A,0.0486633,N/A,-0.00681973,N/A,-0.025354,N/A,N/A
3,AAPL,129.490,N/A,0.859883,N/A,0.488681,N/A,0.0128987,N/A,0.0300651,N/A,N/A
4,ABBV,109.660,N/A,0.302981,N/A,0.174043,N/A,0.185013,N/A,0.15783,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.350,N/A,0.096133,N/A,0.108969,N/A,0.120616,N/A,0.0729337,N/A,N/A
501,ZBH,147.241,N/A,0.0114874,N/A,0.0899938,N/A,0.0698012,N/A,0.0784755,N/A,N/A
502,ZBRA,383.380,N/A,0.489973,N/A,0.370621,N/A,0.470612,N/A,0.0964736,N/A,N/A
503,ZION,45.355,N/A,-0.0813399,N/A,0.166312,N/A,0.329685,N/A,0.39509,N/A,N/A


Here some of the response for Price Return was None which was creating problems in getting percentile scores for Price Return for the individual time period. We first find out the stocks for which we don't have Price Return data.

In [55]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
118,CTL,11.00,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
165,ETFC,51.46,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
326,MYL,16.55,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A
327,NBL,8.55,N/A,None,N/A,None,N/A,None,N/A,None,N/A,N/A


Now we remove stocks these stocks from hqm_dataframe so as to get percentile scores of each time period returns.

In [63]:
hqm_dataframe = hqm_dataframe.drop([118, 165, 326, 327])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.710,N/A,0.433324,N/A,0.289282,N/A,0.200653,N/A,0.0233394,N/A,N/A
1,AAL,16.742,N/A,-0.406301,N/A,-0.122349,N/A,0.211873,N/A,0.452247,N/A,N/A
2,AAP,151.820,N/A,-0.00761468,N/A,0.0486633,N/A,-0.00681973,N/A,-0.025354,N/A,N/A
3,AAPL,129.490,N/A,0.859883,N/A,0.488681,N/A,0.0128987,N/A,0.0300651,N/A,N/A
4,ABBV,109.660,N/A,0.302981,N/A,0.174043,N/A,0.185013,N/A,0.15783,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.350,N/A,0.096133,N/A,0.108969,N/A,0.120616,N/A,0.0729337,N/A,N/A
501,ZBH,147.241,N/A,0.0114874,N/A,0.0899938,N/A,0.0698012,N/A,0.0784755,N/A,N/A
502,ZBRA,383.380,N/A,0.489973,N/A,0.370621,N/A,0.470612,N/A,0.0964736,N/A,N/A
503,ZION,45.355,N/A,-0.0813399,N/A,0.166312,N/A,0.329685,N/A,0.39509,N/A,N/A


# Calculating Momentum Percentiles

Now we will calculate momentum percentile scores for every stocks with respect to the price return during different time durations:

- One-Year Price Return
- Six-Month Price Return
- Three-Month Price Return
- One-Month Price Return

These percentile scores will be later used to calculate HQM score for each stock wherein HQM score for each stock will be the arithmetic mean of the 4 momentum percentile scores for different time duration.

In [64]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe       

0       0.858283
1      0.0199601
2       0.329341
3       0.976048
4       0.782435
         ...    
500     0.508982
501     0.349301
502     0.898204
503     0.219561
504     0.784431
Name: One-Year Return Percentile, Length: 501, dtype: object
0       0.794411
1      0.0339321
2       0.269461
3       0.946108
4        0.58483
         ...    
500     0.413174
501     0.371257
502     0.878244
503      0.56487
504      0.47505
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.648703
1      0.678643
2      0.143713
3      0.185629
4       0.60479
         ...   
500    0.447106
501    0.329341
502    0.958084
503     0.89022
504    0.175649
Name: Three-Month Return Percentile, Length: 501, dtype: object
0       0.293413
1        0.94012
2       0.151697
3       0.323353
4       0.684631
         ...    
500     0.449102
501     0.469062
502     0.522954
503     0.922156
504    0.0638723
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.710,N/A,0.433324,0.858283,0.289282,0.794411,0.200653,0.648703,0.0233394,0.293413,N/A
1,AAL,16.742,N/A,-0.406301,0.0199601,-0.122349,0.0339321,0.211873,0.678643,0.452247,0.94012,N/A
2,AAP,151.820,N/A,-0.00761468,0.329341,0.0486633,0.269461,-0.00681973,0.143713,-0.025354,0.151697,N/A
3,AAPL,129.490,N/A,0.859883,0.976048,0.488681,0.946108,0.0128987,0.185629,0.0300651,0.323353,N/A
4,ABBV,109.660,N/A,0.302981,0.782435,0.174043,0.58483,0.185013,0.60479,0.15783,0.684631,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.350,N/A,0.096133,0.508982,0.108969,0.413174,0.120616,0.447106,0.0729337,0.449102,N/A
501,ZBH,147.241,N/A,0.0114874,0.349301,0.0899938,0.371257,0.0698012,0.329341,0.0784755,0.469062,N/A
502,ZBRA,383.380,N/A,0.489973,0.898204,0.370621,0.878244,0.470612,0.958084,0.0964736,0.522954,N/A
503,ZION,45.355,N/A,-0.0813399,0.219561,0.166312,0.56487,0.329685,0.89022,0.39509,0.922156,N/A


# Calculate HQM Score

Now we will calculate HQM Score for each stock. HQM score for a stock is the artithematic mean of 4 momentum percentile score corresponding to 4 time duration that we calculated in the last section.

We will be using this HQM score to to filte out stocks for high mmentum quantitative investing strategy.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [66]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.710,N/A,0.433324,0.858283,0.289282,0.794411,0.200653,0.648703,0.0233394,0.293413,0.648703
1,AAL,16.742,N/A,-0.406301,0.0199601,-0.122349,0.0339321,0.211873,0.678643,0.452247,0.94012,0.418164
2,AAP,151.820,N/A,-0.00761468,0.329341,0.0486633,0.269461,-0.00681973,0.143713,-0.025354,0.151697,0.223553
3,AAPL,129.490,N/A,0.859883,0.976048,0.488681,0.946108,0.0128987,0.185629,0.0300651,0.323353,0.607784
4,ABBV,109.660,N/A,0.302981,0.782435,0.174043,0.58483,0.185013,0.60479,0.15783,0.684631,0.664172
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.350,N/A,0.096133,0.508982,0.108969,0.413174,0.120616,0.447106,0.0729337,0.449102,0.454591
501,ZBH,147.241,N/A,0.0114874,0.349301,0.0899938,0.371257,0.0698012,0.329341,0.0784755,0.469062,0.379741
502,ZBRA,383.380,N/A,0.489973,0.898204,0.370621,0.878244,0.470612,0.958084,0.0964736,0.522954,0.814371
503,ZION,45.355,N/A,-0.0813399,0.219561,0.166312,0.56487,0.329685,0.89022,0.39509,0.922156,0.649202


# Selecting the 50 Best Momentum Stocks¶

We will now finalise 50 best momentum stocks by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [69]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace= True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index()
hqm_dataframe

/Users/onkar.mishra/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,179,FCX,25.285,N/A,1.1962,0.992016,1.32096,1,0.579443,0.976048,0.319745,0.884232,0.963074
1,23,ALB,140.090,N/A,1.25125,0.994012,0.698142,0.982036,0.491907,0.968064,0.241105,0.806387,0.937625
2,288,LRCX,508.900,N/A,0.892298,0.978044,0.663474,0.978044,0.516537,0.97006,0.221461,0.772455,0.924651
3,462,URI,250.000,N/A,0.568362,0.936128,0.522924,0.958084,0.403615,0.94012,0.292027,0.862275,0.924152
4,324,MU,74.160,N/A,0.555802,0.928144,0.378264,0.888224,0.599507,0.982036,0.345322,0.896208,0.923653
5,29,AMAT,92.970,N/A,0.616263,0.944112,0.48596,0.944112,0.475102,0.962076,0.269028,0.836327,0.921657
6,24,ALGN,533.840,N/A,0.920941,0.98004,0.833126,0.994012,0.70967,0.992016,0.0951237,0.520958,0.871756
7,275,LB,40.752,N/A,1.18417,0.99002,1.06331,0.996008,0.276532,0.824351,0.150877,0.662675,0.868263
8,410,SIVB,368.657,N/A,0.514148,0.912176,0.53587,0.962076,0.393449,0.936128,0.146423,0.654691,0.866267
9,495,XLNX,149.400,N/A,0.639843,0.9501,0.572388,0.97006,0.478751,0.964072,0.109832,0.562874,0.861776


# Calculating the Number of Shares to Buy¶

We'll use portflio size and the share price of a stock to calculate the number of stocks to be calculate the number of stocks that need to be bought in this strategy.

In [70]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [71]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,179,FCX,25.285,775,1.1962,0.992016,1.32096,1,0.579443,0.976048,0.319745,0.884232,0.963074
1,23,ALB,140.090,139,1.25125,0.994012,0.698142,0.982036,0.491907,0.968064,0.241105,0.806387,0.937625
2,288,LRCX,508.900,38,0.892298,0.978044,0.663474,0.978044,0.516537,0.97006,0.221461,0.772455,0.924651
3,462,URI,250.000,78,0.568362,0.936128,0.522924,0.958084,0.403615,0.94012,0.292027,0.862275,0.924152
4,324,MU,74.160,264,0.555802,0.928144,0.378264,0.888224,0.599507,0.982036,0.345322,0.896208,0.923653
5,29,AMAT,92.970,210,0.616263,0.944112,0.48596,0.944112,0.475102,0.962076,0.269028,0.836327,0.921657
6,24,ALGN,533.840,36,0.920941,0.98004,0.833126,0.994012,0.70967,0.992016,0.0951237,0.520958,0.871756
7,275,LB,40.752,481,1.18417,0.99002,1.06331,0.996008,0.276532,0.824351,0.150877,0.662675,0.868263
8,410,SIVB,368.657,53,0.514148,0.912176,0.53587,0.962076,0.393449,0.936128,0.146423,0.654691,0.866267
9,495,XLNX,149.400,131,0.639843,0.9501,0.572388,0.97006,0.478751,0.964072,0.109832,0.562874,0.861776


# Saving the output

Now we will save the 

### Reference: 
https://www.youtube.com/watch?v=xfzGZB4HhEE&ab_channel=freeCodeCamp.org